# Analyzing CIA Factbook Data Using SQL

## Introduction

In this project, we will look to analyze CIA factbook data using SQLite and SQL. CIA factbook includes things like population, population growth, area, and other statistics for all countries on Earth.

## Setup

Let's install ipython-sql

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: done

# All requested packages already installed.



Now let's connect our notebook to the database file, which contains the CIA factbook data

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of Data

The first thing we should do is test if we have connected to the database. One thing we can do is run the below query, which returns information about all te tables in the database we have connected to.

In [3]:
%%sql
SELECT * 
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We can see our facts table, which is the table that contains the CIA Factbook data. Let us select the first 5 rows of the facts table.

In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Let's calculate some summary statistics and look for any outlier countries.

In [5]:
%%sql
SELECT
    MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Exploring Outliers

Something looks a bit fishy in the results. There is a country with a population of 0 and a country with a population over 7.2 billion (the population of the world is a little over 7.2 billion). Let's use subqueries to zoom in on these suspicious outliers.

In [6]:
%%sql
SELECT
    name
FROM facts
WHERE population = (
    SELECT 
        MIN(population) 
    FROM facts
);

 * sqlite:///factbook.db
Done.


name
Antarctica


It looks like the CIA Factbook database considers Antarctica a country, and it has a population of 0 because no one lives there.

In [7]:
%%sql
SELECT
    name
FROM facts
WHERE population = (
    SELECT 
        MAX(population) 
    FROM facts
);

 * sqlite:///factbook.db
Done.


name
World


Our prediction was correct. The CIA factbook has an entry fo the World, which has a total population above 7.2 billion.

## Exploring Average Population and Area

Let's recompute the summary statistics, but exclude the World row.

In [8]:
%%sql
SELECT
    MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


Let's also calculate the averages for population and area.

In [9]:
%%sql
SELECT
    AVG(population),
    AVG(area)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

We will define a densely populated country as a country with above-average population and below-average area.

In [10]:
%%sql
SELECT
    name
FROM facts
WHERE
    population > (
        SELECT 
            AVG(population)
        FROM facts
        WHERE name != 'World'
    )
    AND
    area < (
        SELECT
            AVG(area)
        FROM facts
        WHERE name != 'World'
    )
    AND
    name != 'World';

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


## Further Queries to Explore

### Which country has the most people? Which country has the highest growth rate?

In [11]:
%%sql
SELECT
    name, population
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts
    WHERE name != 'World'
);

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


In [12]:
%%sql
SELECT
    name, population_growth
FROM facts
WHERE population_growth = (
    SELECT MAX(population_growth)
    FROM facts
    WHERE name != 'World'
);

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [13]:
%%sql
SELECT
    name, 
    ROUND(CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT), 4) AS water_to_land_ratio
FROM facts
ORDER BY water_to_land_ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,water_to_land_ratio
British Indian Ocean Territory,905.6667
Virgin Islands,4.5202
Puerto Rico,0.5548
"Bahamas, The",0.3866
Guinea-Bissau,0.2847
Malawi,0.2594
Netherlands,0.2257
Uganda,0.2229
Eritrea,0.1644
Liberia,0.1562


In [14]:
%%sql
SELECT
    name, 
    area_water,
    area_land
FROM facts
WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


### Which countries will add the most people to their populations next year?

In [15]:
%%sql
SELECT
    name,
    population_growth
FROM facts
ORDER BY population_growth DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


### Which countries have a higher death rate than birth rate?

In [16]:
%%sql
SELECT
    name,
    birth_rate,
    death_rate
FROM facts
WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


## Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [17]:
%%sql
SELECT
    name,
    ROUND(CAST(population AS FLOAT) / CAST(area AS FLOAT)) AS population_to_area_ratio
FROM facts
ORDER BY population_to_area_ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population_to_area_ratio
Macau,21169.0
Monaco,15268.0
Singapore,8141.0
Hong Kong,6445.0
Gaza Strip,5192.0
Gibraltar,4876.0
Bahrain,1772.0
Maldives,1320.0
Malta,1310.0
Bermuda,1300.0


The countries returned in this query are different from the countries returned in our "high density" query, mainly because the populations of these countries are all below the world average. However, because they are all extremely small in area as well, they have the highest population densities in the world.